In [2]:
# %load_ext autoreload
# %autoreload 2
import os, sys
sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.getcwd())) 

import numpy as np
import pandas as pd
import torch
from scipy.special import softmax
from sklearn.preprocessing import PolynomialFeatures


from core.conformal import compute_conformity_score, compute_conformity_score_softmax, compute_conformity_score_aps, compute_conformity_score_raps, compute_sets_split, compute_sets_cond
from core.evaluation import aggregate_results_over_seeds
from utils.model import get_image_classifier, split_test
from utils.data import get_image_dataset

### Define Your Custom Function Class

Create your custom function class that will generate features for conditional conformal prediction. The class should have a `compute_features` method that takes model outputs and returns features.

In [ ]:
class CustomFunctionClass:
    def __init__(self):
        """Initialize your custom function class"""
        pass
        
    def compute_features(self, logits, features=None):
        """Compute features for conditional conformal prediction
        
        Args:
            logits: Model output logits of shape (n_samples, n_classes)
            features: Optional model features of shape (n_samples, feature_dim)
            
        Returns:
            phi: Features for conditional conformal prediction of shape (n_samples, n_features)
        """
        # Example: Use maximum softmax probability as a feature
        probs = softmax(logits, axis=1)
        max_probs = np.max(probs, axis=1, keepdims=True)
        return max_probs

### Configuration

In [ ]:
# Set paths and parameters
DATA_DIR = '../data'
CACHE_DIR = '~/.cache'
FEATURES_DIR = '../features'
OUTPUT_DIR = '../outputs'

# Model and dataset parameters
dataset_name = 'imagenet'  
model_name = 'resnet50'    
batch_size = 64

# Algorithm parameters
alpha = 0.1              
score_fn = 'aps'        
scores_randomize = False  
temp_scaling = True                  
seed = 1                 

### Load Model and Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load model
model, preprocess = get_image_classifier(model_name, device=device)

# Load dataset and compute/load features
if dataset_name in ["imagenet", "places"]:
    train_dataset, test_dataset = get_image_dataset(dataset_name, preprocess=preprocess)
    test_features, test_logits, test_labels = model.run_and_cache_outputs(test_dataset, batch_size, FEATURES_DIR)
    test_labels, test_features, test_logits, _, calib_labels, calib_features, calib_logits, _ = split_test(
        test_labels, test_features, test_logits, split=0.5, seed=seed
    )
else:
    train_dataset, val_dataset, test_dataset = get_image_dataset(dataset_name, preprocess=preprocess)
    calib_features, calib_logits, calib_labels = model.run_and_cache_outputs(val_dataset, batch_size, FEATURES_DIR)
    test_features, test_logits, test_labels = model.run_and_cache_outputs(test_dataset, batch_size, FEATURES_DIR)

### Compute Features using Custom Function Class

In [ ]:
# Initialize your custom function class
custom_function = CustomFunctionClass()

# Compute features for calibration and test sets
calib_phi = custom_function.compute_features(calib_logits, calib_features)
test_phi = custom_function.compute_features(test_logits, test_features)

### Compute conformity scores and prediction sets

In [ ]:
# Compute conformity scores
if score_fn == "softmax":
    calib_scores, test_scores, test_scores_all = compute_conformity_score_softmax(
        calib_logits, test_logits, calib_labels, test_labels, temp_scaling=temp_scaling
    )
elif score_fn == "aps":
    calib_scores, test_scores, test_scores_all = compute_conformity_score_aps(
        calib_logits, test_logits, calib_labels, test_labels, rand=scores_randomize, temp_scaling=temp_scaling
    )
elif score_fn == "raps":
    calib_scores, test_scores, test_scores_all = compute_conformity_score_raps(
        calib_logits, test_logits, calib_labels, test_labels, rand=scores_randomize, temp_scaling=temp_scaling
    )

# Compute prediction sets for split conformal
coverages_split, prediction_sets_split, set_sizes_split = compute_sets_split(
    calib_scores, test_scores, test_scores_all, alpha
)

# Compute prediction sets for conditional conformal
coverages_cond, prediction_sets_cond, set_sizes_cond = compute_sets_cond(
    calib_phi, calib_scores, test_phi, test_scores, test_scores_all, alpha, rand=scores_randomize
)

### Save results

In [ ]:
# Setup output directory and save results
results_dir = os.path.join(OUTPUT_DIR, f"{dataset_name}_{model_name}")
os.makedirs(results_dir, exist_ok=True)

res_fname = f"alpha_{alpha}_score_fn_{score_fn}_scores_randomize_{scores_randomize}_temp_scale_{temp_scaling}_seed_{seed}"

np.save(os.path.join(results_dir, f"{res_fname}_calib_phi.npy"), calib_phi)
np.save(os.path.join(results_dir, f"{res_fname}_test_phi.npy"), test_phi)
np.save(os.path.join(results_dir, f"{res_fname}_coverages_split.npy"), coverages_split)
np.save(os.path.join(results_dir, f"{res_fname}_prediction_sets_split.npy"), prediction_sets_split)
np.save(os.path.join(results_dir, f"{res_fname}_set_sizes_split.npy"), set_sizes_split)
np.save(os.path.join(results_dir, f"{res_fname}_coverages_cond.npy"), coverages_cond)
np.save(os.path.join(results_dir, f"{res_fname}_prediction_sets_cond.npy"), prediction_sets_cond)
np.save(os.path.join(results_dir, f"{res_fname}_set_sizes_cond.npy"), set_sizes_cond)